In [8]:
import pennylane as qml
import numpy as np
from pennylane.optimize import AdamOptimizer
import torch

In [9]:
def layer(varr):
    qml.Rotation(varr[0], wires=0)
    qml.Squeezing(varr[1], 0., wires=0)
    qml.Rotation(varr[2], wires=0)
    qml.Displacement(varr[3], 0., wires=0)
    qml.Kerr(varr[4], wires=0)

In [10]:
def SquareLoss(labelValues, predictionValues):
    lossValue = 0
    for label, prediction in zip(labelValues, predictionValues):
        #print(label,prediction)
        lossValue = lossValue + (label - prediction) ** 2
    lossValue = lossValue / len(labelValues)
    return lossValue

In [11]:
device = qml.device('strawberryfields.fock', wires=1,cutoff_dim=10)
@qml.qnode(device)
def QuantumNeuralNet(vars1, xcoor=None):
    qml.Displacement(xcoor, 0., wires=0)
    #print(xcoor)
    for var in vars1:
        layer(var)
    out=qml.expval(qml.X(0))
    #print(out)
    return out

In [12]:
def Cost(variables, featureValues, labelValues):
    predictions = [QuantumNeuralNet(variables, xcoor=x) for x in featureValues]
    #print(predictions)
    return SquareLoss(labelValues, predictions)
    #loss=torch.abs(torch.Tensor(predictions) - torch.Tensor(labelValues)) ** 2
    #loss.backward()
    #return 

In [13]:
number_of_layers = 1
variable_initial = 0.05 * np.random.randn(number_of_layers, 5)
# getting the adam optimizer
optimizer = AdamOptimizer(0.01, beta1=0.9, beta2=0.999)

In [14]:
X_coord = [i for i in range(1,100)]
Y_coord = [np.sin(np.pi*i/30) for i in range(1,100)]
# variable set to variable_intial
variable = variable_initial
#iterating from 1 to 500
for iteration in range(5):
    var_cost = optimizer.step(lambda v: Cost(v, X_coord, Y_coord), variable)
    print(variable)
    # printing the iteration, cost value
    print("Iteration: {:5d} | Cost value: {:0.7f} ".format(iteration + 1, Cost(var_cost, X_coord, Y_coord)))

[[ 0.02844443  0.10071516  0.04005486 -0.02087672  0.02569614]]
Iteration:     1 | Cost value: 0.6419434 
[[ 0.02844443  0.10071516  0.04005486 -0.02087672  0.02569614]]
Iteration:     2 | Cost value: 0.6434084 
[[ 0.02844443  0.10071516  0.04005486 -0.02087672  0.02569614]]
Iteration:     3 | Cost value: 0.6440086 
[[ 0.02844443  0.10071516  0.04005486 -0.02087672  0.02569614]]
Iteration:     4 | Cost value: 0.6443365 
[[ 0.02844443  0.10071516  0.04005486 -0.02087672  0.02569614]]
Iteration:     5 | Cost value: 0.6445389 


In [15]:
def predict(var,x):
    preds=QuantumNeuralNet(var, xcoor=x)
    return preds

In [16]:
predict(variable,np.pi/2)

2.7350793484415474